In [1]:
import numpy 
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


Using TensorFlow backend.


In [2]:
from Bio import SeqIO
from Bio.Data import IUPACData 

def protein2integer(in_seq):
    
    ## define universe of possible input values
    all_protein_letters = list(IUPACData.extended_protein_letters)
    #print(all_protein_letters)
    ## define a mapping of chars to integers
    char_to_int = dict((c, i) for i, c in enumerate(all_protein_letters))
    int_to_char = dict((i, c) for i, c in enumerate(all_protein_letters))
    ## integer encode input data
    integer_encoded = [char_to_int[char] for char in in_seq.upper()]
    
    #return(integer_encoded,len(all_protein_letters))
    return(integer_encoded)
    

In [3]:
import csv
import numpy as np
data_path = 'features.csv'
with open(data_path, 'r') as f:
    reader = csv.reader(f, delimiter="\t")
    # get all the rows as a list
    dataset = list(reader)
    # transform data into numpy array
    dataset = np.array(dataset).astype(str)

print(dataset[:,0])
print(dataset[0:2,1])
print(dataset[:,2].astype(int))
print(dataset.shape)

['MARPO07937' 'ORYBR13189' 'ORYGL12366' 'ORYLO20560' 'ORYNI16379'
 'ORYRU16587' 'ORYSI15948' 'ORYSJ14856' 'BRADI07563' 'HORVV106231'
 'AEGTA03913' 'WHEAT40487' 'TRIUA17650' 'ERATE34499' 'SORBI12540'
 'MAIZE00159' 'SETIT35121' 'MUSAC02596' 'MUSAM06245' 'CHEQI00829'
 'NICAT07910' 'SOLTU21094' 'SOYBN32269' 'MEDTR35254' 'POPTR21272'
 'PRUPE14530' 'ARAAL19814' 'BRARP07873' 'BRANA06689' 'ARALY22171'
 'ARATH34990' 'THECC11816' 'GOSHI65213' 'ORYNI25992' 'ORYPU22376'
 'ORYRU26544' 'ORYSI26003' 'ORYSJ23753' 'BRADI09931' 'HORVV192917'
 'WHEAT23509' 'ERATE16670' 'MAIZE31955' 'SETIT10568' 'MUSAC31008'
 'MUSAM27905' 'SOLLC02705' 'SOLTU04879' 'LUPAN08914' 'MANES15049'
 'POPTR16484' 'PRUPE13257' 'BRANA90718' 'BRAOL47337' 'ARALY00095'
 'ARATH00066' 'THECC03451' 'ORYBR16014' 'ORYGL27982' 'ORYLO22608'
 'ORYNI36636' 'ORYPU31377' 'ORYRU37270' 'ORYSI36479' 'ORYSJ33184'
 'HORVV130178' 'AEGTA25528' 'WHEAT33067' 'ERATE21849' 'SORBI01205'
 'MAIZE11545' 'CHEQI00389' 'SOLLC13604' 'BRANA38027' 'BRAOL53590'
 'GOSHI

In [4]:
print(dataset[:,1].shape)
integer_encoded_proteins = np.array([protein2integer(seq) for seq in dataset[:,1]])
len(integer_encoded_proteins)
integer_encoded_proteins[0]
#np.array(integer_encoded_proteins).shape
integer_encoded_proteins.shape
#protein2integer(dataset[:,1])

(76,)


(76,)

In [5]:
# Loading the input values to X and Label values Y using slicing.
G = dataset[:, 0]
X = integer_encoded_proteins
Y = dataset[:, 2].astype(int)

print(G.shape)
print(X.shape)
print(Y.shape)

print(G[0:3,])
print(X[0:3,])
print(Y[0:3,])

(76,)
(76,)
(76,)
['MARPO07937' 'ORYBR13189' 'ORYGL12366']
[list([10, 0, 0, 10, 16, 10, 16, 14, 12, 11, 2, 12, 4, 16, 15, 2, 7, 0, 14, 4, 6, 3, 17, 8, 5, 0, 8, 2, 3, 12, 4, 0, 3, 9, 16, 14, 19, 6, 3, 17, 14, 5, 17, 14, 3, 10, 17, 3, 15, 4, 14, 17, 14, 3, 17, 12, 15, 19, 19, 17, 8, 12, 17, 11, 3, 14, 14, 4, 16, 12, 12, 15, 0, 0, 17, 9, 15, 10, 3, 13, 13, 7, 12, 1, 7, 2, 9, 3, 0, 9, 15, 5, 13, 3, 9, 9, 15, 0, 7, 0, 11, 0, 1, 14, 2, 18, 5, 4, 4, 13, 17, 9, 11, 6, 5, 9, 12, 15, 13, 9, 17, 13, 11, 10, 0, 8, 13, 15, 15, 3, 4, 4, 0, 13, 12, 9, 3, 3, 8, 10, 8, 1, 15, 16, 12, 0, 14, 17, 15, 5, 12, 17, 6, 4, 5, 5, 5, 5, 11, 14, 2, 18, 14, 2, 17, 9, 8, 9, 11, 1, 0, 12, 0, 15, 7, 17, 0, 8, 3, 19, 18, 12, 13, 14, 12, 0, 5, 4, 14, 2, 16, 10, 3, 3, 19, 15, 15, 13, 13, 13, 0, 9, 0, 7, 14, 9, 9, 8, 9, 7, 15, 3, 15, 9, 5, 9, 3, 15, 11, 19, 9, 17, 0, 0, 1, 5, 3, 12, 8, 17, 17, 10, 0, 7, 11, 6, 19, 12, 12, 1, 12, 2, 12, 15, 9, 16, 10, 5, 7, 8, 0, 6, 15, 2, 12, 11, 16, 7, 16, 10, 9, 9, 13, 2, 2, 17, 5, 5, 

In [6]:
print('Maximum review length: {}'.format(len(max(X, key=len))))
print('Minimum review length: {}'.format(len(min(X, key=len))))
print(type(X))
print(X.shape)
print(Y.shape)
print(G.shape)

indices = numpy.random.permutation(X.shape[0])
train_size = int(indices.size*0.8)
train_idx, test_idx = indices[:train_size], indices[train_size:]
print(len(train_idx),len(test_idx))
X_train, X_test = X[train_idx,], X[test_idx,]
print(X_train.shape,X_test.shape)
y_train, y_test = Y[train_idx,], Y[test_idx,]
G_train, G_test = G[train_idx,], G[test_idx,]

print(G[train_idx[0],])
print(X[train_idx[0],])
print(Y[train_idx[0],])

print(G_train[0,])
print(X_train[0,])
print(y_train[0,])

Maximum review length: 456
Minimum review length: 122
<type 'numpy.ndarray'>
(76,)
(76,)
(76,)
(60, 16)
((60,), (16,))
ARAAL19814
[10, 0, 5, 8, 9, 7, 15, 16, 5, 9, 14, 9, 16, 16, 9, 12, 5, 11, 19, 17, 14, 12, 4, 15, 2, 14, 12, 14, 9, 15, 3, 17, 15, 13, 9, 3, 2, 4, 12, 9, 7, 2, 9, 15, 15, 16, 2, 14, 15, 14, 9, 17, 13, 13, 7, 6, 13, 0, 1, 16, 14, 4, 5, 4, 4, 13, 17, 7, 11, 6, 5, 17, 2, 8, 3, 16, 7, 2, 3, 10, 17, 15, 17, 0, 11, 3, 4, 4, 15, 10, 15, 10, 3, 3, 8, 10, 8, 9, 19, 15, 2, 2, 12, 16, 14, 16, 16, 14, 9, 15, 16, 15, 4, 11, 17, 8, 8, 3, 3, 7, 11, 11, 18, 14, 2, 19, 9, 14, 9, 6, 1, 19, 12, 7, 2, 8, 19, 7, 6, 3, 18, 12, 15, 11, 12, 7, 12, 4, 8, 11, 17, 17, 15, 8, 19, 15, 14, 3, 17, 14, 3, 17, 5, 4, 14, 7, 3, 3, 9, 7, 15, 3, 15, 9, 5, 9, 3, 16, 2, 19, 9, 14, 8, 17, 9, 5, 3, 13, 5, 13, 6, 10, 0, 17, 11, 19, 19, 12, 12, 1, 12, 3, 12, 3, 9, 16, 19, 5, 9, 12, 0, 6, 16, 2, 12, 11, 0, 9, 16, 7, 9, 9, 13, 2, 15, 16, 17, 1, 5, 9, 13, 7, 9, 7, 2, 5, 13, 18, 4, 0, 7, 11, 12, 6, 12, 2, 0, 4, 17, 

In [11]:
# https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/
# truncate and pad input sequences
top_words = len(list(IUPACData.extended_protein_letters)) # = 26
max_review_length = len(max(X, key=len)) # = 500
X_train_new = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test_new = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train_new, y_train, epochs=50, batch_size=10) #epochs=3, batch_size=64)
# Final evaluation of the model
scores = model.evaluate(X_test_new, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 456, 32)           832       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 456, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 228, 32)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 101       
Total params: 57,237
Trainable params: 57,237
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
60/60 [==============================] - 2s 29ms/step - loss: 0.6538 - acc: 0.4500
Epoch 2/50
60/60 [=====================

In [8]:
print(X_train_new.shape)
print(X_train_new[0,:])
print(X_train.shape)
print(X_train[0,])


(60, 456)
[ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 10  0  5  8  9
  7 15 16  5  9 14  9 16 16  9 12  5 11 19 17 14 12  4 15  2 14 12 14  9
 15  3 17 15 13  9  3  2  4 12  9  7  2  9 15 15 16  2 14 15 14  9 17 13
 13  7  6 13  0  1 16 14  4  5  4  4 13 17  7 11  6  5 17  2  8  3 16  7
  2  3 10 17 15 17  0 11  3  4  4 15 10 15 10  3  3  8 10  8  9 19 15  2
  2 12 16 14 16 16 14  9 15 16 15  4 11 17  8  8  3  3  7 11 11 18 14  2
 19  9 14  9  6  1 19 12  7  2  8 19  7  6  3 18 12 15 11 12  7 12  4  8
 11 17 17 15  8 19 15 14  3 17 14  3 17  5  4 14  7  3  3  9  7 15  3 15
  9  5  9  3 16  2 19  9 14  8 17  9  5  3 13  5 13  6 10  0 17 11 19 19
 12 12  1 12  3 12  3  9 16 19  5  9 12  

In [10]:
# Initializing the Sequential model from KERAS.
model2 = Sequential()

max_review_length = len(max(X, key=len))

# Creating a 16 neuron hidden layer with Linear Rectified activation function.
#model.add(Dense(16, input_dim=1, init='uniform', activation='relu'))
model2.add(Dense(16, input_dim=max_review_length, kernel_initializer='uniform', activation='relu'))

# Creating a 8 neuron hidden layer.
model2.add(Dense(8, kernel_initializer='uniform', activation='relu'))

# Adding a output layer.
model2.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))



# Compiling the model
model2.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])

print(model2.summary())

# Fitting the model
#model.fit(X, Y, nb_epoch=150, batch_size=10)
#scores = model.evaluate(X, Y)
model2.fit(X_train_new, y_train, epochs=50, batch_size=10)
scores = model2.evaluate(X_test_new, y_test, verbose=0)

print("%s: %.2f%%" % (model2.metrics_names[1], scores[1] * 100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 16)                7312      
_________________________________________________________________
dense_6 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 9         
Total params: 7,457
Trainable params: 7,457
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
60/60 [==============================] - 0s 6ms/step - loss: 0.5922 - acc: 0.4500
Epoch 2/50
60/60 [==============================] - 0s 172us/step - loss: 0.2883 - acc: 0.4500
Epoch 3/50
60/60 [==============================] - 0s 180us/step - loss: -0.0962 - acc: 0.4500
Epoch 4/50
60/60 [==============================] - 0s 181us/step - loss: -0.5720 - acc: 0.4500
Epoch 5/50
60/60 [=